In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install biopython scikit-learn --quiet
!pip install biopython scikit-learn tensorflow --quiet

import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from random import shuffle

# Function to load sequences from FASTA files
def load_sequences(file_paths, window=150, stride=50):
    sequences = []
    labels = []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)

# Function to generate k-mers
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

# Build k-mer frequency matrix
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df

# File paths
file_paths = {
    'Zika': '/content/drive/MyDrive/Datasets/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/Datasets/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/Datasets/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/Datasets/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/Datasets/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/Datasets/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/Datasets/Human_200.fasta'
}

# Load and process data
data, labels = load_sequences(file_paths, window=150, stride=50)
print(f"\nAugmented sequences: {len(data)}")

X = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]

# Feature selection
selector = SelectKBest(mutual_info_classif, k=160)
X_selected = selector.fit_transform(X, y)
selected_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_indices]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert labels to categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Build DNN model
model = Sequential([
    Dense(300, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(y_train_cat.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train_cat, epochs=15, batch_size=64, verbose=0)

# Predict and evaluate
y_pred_probs = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nKeras DBN Accuracy:", accuracy_score(y_test, y_pred) * 100)
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))
print("\nSelected Features (Top 160 k-mers):")
print(selected_features.tolist())



Augmented sequences: 9820


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

Keras DNN Accuracy: 98.11608961303462

Classification Report:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       829
           1       0.96      0.82      0.89        67
           2       0.99      0.99      0.99       252
           3       0.99      0.97      0.98       426
           4       0.94      0.99      0.96       237
           5       1.00      0.99      1.00       151
           6       1.00      0.50      0.67         2

    accuracy                           0.98      1964
   macro avg       0.98      0.89      0.92      1964
weighted avg       0.98      0.98      0.98      1964


Selected Features (Top 160 k-mers):
['AAA', 'AAD', 'AAQ', 'AAS', 'AAT', 'AEA', 'AER', 'AFF', 'AFG', 'AGA', 'AGG', 'AGP', 'AII', 'ANL', 'API', 'AQM', 'ARI', 'ARL', 'ASP', 'ATL', 'CYP', 'DAL', 'DDP', 'DEL', 'DGA', 'DGD', 'DHA', 'DHS', 'DHT', 'DLV', 'DPD', 'DRS', 'DVH', 'EFI', 'EHR', 'ELH', 'ELL', 'ES